# Recommender System for Board Games from [BoardGameGeek.com](https://boardgamegeek.com/)



1.   Iatrou Manos
2.   Papageorgiou Vasileios
3. Sykianakis Xaralambos




# Dataset Description



*   Games File
*   User Ratings File
*   Mechanics File
*   Themes File



In [72]:
!pip install alibi

     -------------------------------------- 524.2/524.2 kB 3.6 MB/s eta 0:00:00
     ---------------------------------------- 22.8/22.8 MB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 4.3/4.3 MB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 98.5/98.5 MB 5.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.22.0
    Uninstalling scikit-image-0.22.0:
      Successfully uninstalled scikit-image-0.22.0


In [77]:
import warnings
warnings.filterwarnings('ignore', category=NumbaDeprecationWarning)

NameError: name 'NumbaDeprecationWarning' is not defined

In [73]:
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
import lightgbm
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping, log_evaluation
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.cluster import DBSCAN
import umap
import optuna
from anchor import anchor_tabular
from alibi.explainers import AnchorTabular
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", message=".*colsample_bytree.*")
warnings.filterwarnings("ignore", message=".*subsample.*")

#from google.colab import drive
#drive.mount('/content/drive')

C:\Users\maniat\AppData\Local\anaconda3\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
C:\Users\maniat\AppData\Local\anaconda3\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_orde

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.


In [32]:
def get_description(df: pd.DataFrame):
    # accepts a dataframe and collapses many one hot encoded columns to 
    # text with column name if 1
    cols = df.columns[15:]
    categories = df[cols].gt(0).apply(lambda x: x.index[x].tolist(), axis=1)
    
    return categories.apply(lambda x: ','.join(x) if x else 'Other')

In [33]:
def downcasting_types(df: pd.DataFrame):
  for column in df.columns:
    colType = df[column].dtype
    if colType == 'float64' :
      df[column] = pd.to_numeric(df[column], downcast='float')
    elif colType == 'int64' :
      df[column] = pd.to_numeric(df[column], downcast='integer')
  return df

In [34]:
def load_data(path: str):
    """
    Load all CSV files in the path folder and return a dictionary of DataFrames.

    """
    folder = Path(path)
    if not folder.exists() or not folder.is_dir():
        raise FileNotFoundError(f"{path} directory was not found")

    dataframes = {}
    for file in folder.glob('*.csv'):
        file_name = file.stem
        df = pd.read_csv(file)
        dataframes[file_name] = df

    return dataframes


In [56]:
def preprocess_and_merge(dataframes: dict):
    """
    Preprocess and merge DataFrames from the given dictionary
    and returns a tuple ,(user_ratings, games_df).
    """
    columns_to_discretize = ['GameWeight', 'MfgPlaytime']
    columns_to_keep = ['BGGId', 'GameWeight', 'MfgPlaytime', 'NumAlternates', 'NumExpansions', 
                               'NumImplementations', 'Kickstarted', 'Cat:Thematic', 'Cat:Strategy', 'Cat:War', 
                               'Cat:Family', 'Cat:CGS', 'Cat:Abstract', 'Cat:Party', 'Cat:Childrens']
    for file_name, df in dataframes.items():
        if file_name == "games":
            # Preprocessing for 'games'
            df = df[columns_to_keep].copy()
            renaming_dict = {col: col.replace('Cat:', '') if col.startswith('Cat:')
                             else col
                             for col in df.columns}
            df.rename(columns=renaming_dict, inplace=True)
            discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
            df[columns_to_discretize] = discretizer.fit_transform(df[columns_to_discretize])
            
            
        elif file_name == "themes":
            # Preprocessing for 'themes'
            renaming_dict = {col: col.replace('Theme_', '') if col.startswith('Theme_') 
                             else col
                             for col in df.columns}
            df.rename(columns=renaming_dict, inplace=True)
        
        elif file_name == "mechanics":
            pass

        elif file_name == "user_ratings":
            
            df['Rating'] = df['Rating'].apply(lambda x: 1 if x > 6 else -1)
            label_encoder = LabelEncoder()
            df['uid'] = label_encoder.fit_transform(df['Username'])
            df.drop('Username', axis=1, inplace=True)
            df = df.groupby('uid').filter(lambda x: len(x) >= 300).groupby('BGGId').filter(lambda x: len(x) >= 100)
            df.drop_duplicates(subset=['uid', 'BGGId'], keep='last', inplace=True)

        dataframes[file_name] = df

    # Prepare final games dataframe
    merged_df = pd.merge(dataframes['games'], dataframes['mechanics'], on='BGGId', how='inner')
    games_df = pd.merge(merged_df, dataframes['themes'], on='BGGId', how='inner')
    games_df['Details'] = get_description(games_df)
    columns_to_drop = dataframes['themes'].columns[1:].tolist() + dataframes['mechanics'].columns[1:].tolist()
    games_df = games_df.drop(columns=columns_to_drop, axis=1)
    games_df = downcasting_types(games_df) 
    
    
    # Prepare final user_ratings dataframe
    user_ratings_df = dataframes.get('user_ratings', pd.DataFrame())
    user_ratings_df = downcasting_types(user_ratings_df)
     

    return user_ratings_df, games_df



In [57]:
def create_user_clusters(train_df: pd.DataFrame,
                         test_df: pd.DataFrame,
                         uid_col: str, #user id col
                         gameid_col: str, #game id col
                         rating_col: str): #rating col
    """
    Create user clusters for both training and test data
    using UMAP for dimensionality reduction and DBSCAN for clustering
    and returns a tuple of the clusters for the X_train and X_test

    """

    #
    user_game_matrix_train = train_df.pivot(index=uid_col, columns=gameid_col, values=rating_col).fillna(0)

    # UMAP to reduce dimensions on training data
    reduction = umap.UMAP(n_neighbors=20, n_components=100, metric='cosine', min_dist=0.0, random_state=42)
    embedding_train = reduction.fit_transform(user_game_matrix_train)

    # DBSCAN on training data
    dbscan = DBSCAN(eps=0.26, min_samples=15, metric='euclidean')
    clusters_train = dbscan.fit_predict(embedding_train)
    user_game_matrix_train['Cluster'] = clusters_train

    # Transform test data using the trained UMAP model
    user_game_matrix_test = test_df.pivot(index=uid_col, columns=gameid_col, values=rating_col).fillna(0)
    embedding_test = reduction.transform(user_game_matrix_test)
    clusters_test = dbscan.fit_predict(embedding_test)
    user_game_matrix_test['Cluster'] = clusters_test

    return user_game_matrix_train[['Cluster']].reset_index(), user_game_matrix_test[['Cluster']].reset_index()


In [58]:
def prepare_train_test_split(path: str):
    """
    Loads data, preprocesses it, merges, and returns 
    the train, test dataframes.
    """
    # Call load_data, preprocess_and_merge
    dataframes = load_data(path)
    user_ratings_df, games_df = preprocess_and_merge(dataframes)

   
    X = pd.merge(user_ratings_df, games_df, on='BGGId', how='inner')
    y_model = X['Rating'] 
    X_model = X.drop(['Details'], axis=1)  

    X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.20)

    train_for_clustering = X_train[['uid', 'BGGId', 'Rating']]
    test_for_clustering = X_test[['uid', 'BGGId', 'Rating']]

    user_clusters_train, user_clusters_test = create_user_clusters(train_for_clustering, test_for_clustering, 'uid', 'BGGId', 'Rating')

    X_train = X_train.merge(user_clusters_train, on='uid', how='left')
    X_test = X_test.merge(user_clusters_test, on='uid', how='left')

    X_train = X_train.drop(['Rating'], axis=1)
    X_test = X_test.drop(['Rating'], axis=1)
    y_train = y_train.replace(-1, 0)
    y_test = y_test.replace(-1, 0)

    return X_train, X_test, y_train, y_test


In [59]:
def optimize_hyperparameters(X_train: pd.DataFrame,
                             y_train: pd.DataFrame,
                             n_trials: int = 5,
                             cv_folds: int = 5):
    """
    Performs hyperparameter optimization using Optuna for LGBMClassifier.
    Uses X_train and y_train for n_trials and returns the best 
    parameters of the optimization.
    """
    
    study = optuna.create_study(direction="maximize")
    
    def objective(trial):
        params = {
            "boosting_type": "gbdt",
            "num_leaves": trial.suggest_int("num_leaves", 2, 256),
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.5),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.1, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.1, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
            "min_child_samples": trial.suggest_int("min_child_samples", 1, 100)
        }

       
        model = LGBMClassifier(objective='binary',
                               metric='binary_logloss',
                               **params, n_jobs=-1)

        # CV
        scores = cross_val_score(model, X_train, y_train,
                                 cv=cv_folds, scoring='roc_auc')
        return scores.mean()

    study.optimize(objective, n_trials=n_trials)
    best_params = study.best_params
    return best_params

In [60]:

def train_model_with_evaluation(X_train: pd.DataFrame, 
                                y_train: pd.DataFrame,
                                X_test: pd.DataFrame,
                                y_test: pd.DataFrame,
                                best_params: dict,
                                n_estimators: int  # number of boosting rounds for LGBM
                               ):
    """
    Trains the model using the best parameters found 
    by Optuna and evaluates it on the test set and returns the 
    trained LightGBM model.
    """
    # Create dictionary for params
    best_params["objective"] = "binary"
    best_params["metric"] = "binary_logloss"
    best_params["n_estimators"] = n_estimators
    best_params["n_jobs"] = -1

    
    model = LGBMClassifier(**best_params)

   
    early_stopping_callback = early_stopping(stopping_rounds=100, first_metric_only=True, verbose=True)
    log_evaluation_callback = log_evaluation(period=50)

   
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], callbacks=[early_stopping_callback, log_evaluation_callback])

    
    y_pred = model.predict_proba(X_test)[:, 1]

    
    roc_auc = roc_auc_score(y_test, y_pred)
    print("ROC AUC on the test set:", roc_auc)

    return model, roc_auc


In [61]:
folder_path = r'C:\Users\maniat\OneDrive - Lyse AS\Desktop\personal\MSc\Customer Analytics\Recommender Classification\Recommender_Files'
X_train, X_test, y_train, y_test = prepare_train_test_split(folder_path)

C:\Users\maniat\AppData\Local\anaconda3\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [65]:
opt_params = optimize_hyperparameters(X_train, y_train, n_trials=2)
trained_model,roc_auc = train_model_with_evaluation(X_train, y_train, X_test, y_test, best_params = opt_params, n_estimators=10)




[I 2024-01-20 00:36:49,292] A new study created in memory with name: no-name-ef01ed6b-d565-4515-bda3-a3713003f2ff


[LightGBM] [Warning] feature_fraction is set=0.7790032180615114, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7790032180615114
[LightGBM] [Warning] bagging_fraction is set=0.46392871527854584, subsample=1.0 will be ignored. Current value: bagging_fraction=0.46392871527854584
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.7790032180615114, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7790032180615114
[LightGBM] [Warning] bagging_fraction is set=0.46392871527854584, subsample=1.0 will be ignored. Current value: bagging_fraction=0.46392871527854584
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.7790032180615114, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7790032180615114
[LightGBM] [Warning] bag

[I 2024-01-20 00:40:19,036] Trial 0 finished with value: 0.7488734072732074 and parameters: {'num_leaves': 135, 'learning_rate': 0.41119433145044537, 'feature_fraction': 0.7790032180615114, 'bagging_fraction': 0.46392871527854584, 'bagging_freq': 6, 'min_child_samples': 32}. Best is trial 0 with value: 0.7488734072732074.


[LightGBM] [Warning] feature_fraction is set=0.9935014323558218, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9935014323558218
[LightGBM] [Warning] bagging_fraction is set=0.8524738589877902, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8524738589877902
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] feature_fraction is set=0.9935014323558218, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9935014323558218
[LightGBM] [Warning] bagging_fraction is set=0.8524738589877902, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8524738589877902
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] feature_fraction is set=0.9935014323558218, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9935014323558218
[LightGBM] [Warning] bagging

[I 2024-01-20 00:47:05,034] Trial 1 finished with value: 0.744549797163661 and parameters: {'num_leaves': 245, 'learning_rate': 0.06759016136704075, 'feature_fraction': 0.9935014323558218, 'bagging_fraction': 0.8524738589877902, 'bagging_freq': 9, 'min_child_samples': 84}. Best is trial 0 with value: 0.7488734072732074.


[LightGBM] [Warning] feature_fraction is set=0.7790032180615114, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7790032180615114
[LightGBM] [Warning] bagging_fraction is set=0.46392871527854584, subsample=1.0 will be ignored. Current value: bagging_fraction=0.46392871527854584
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	valid_0's binary_logloss: 0.596069
Evaluated only: binary_logloss
ROC AUC on the test set: 0.6843827604384857


In [ ]:
def user_game_predict (X_train: pd.DataFrame,
                       X_test: pd.DataFrame,
                       model # LGBM model
                    ): 
    
    column_order = X_train.columns.tolist()
    dataset = pd.concat([X_train,X_test])
    new_user = dataset.drop_duplicates(subset='uid').loc[:, ['uid', 'Cluster']]
    new_game = dataset.drop_duplicates(subset='BGGId').drop(columns=['uid', 'Cluster'])
    #pick random user,game
    random_user = new_user['uid'].sample(n=1).iloc[0]
    random_game = new_game['BGGId'].sample(n=1).iloc[0]
    # filter dataframes
    selected_user = new_user[new_user['uid'] == random_user]
    selected_game = new_game[new_game['BGGId'] == random_game]
    test_instance = pd.concat([selected_user.reset_index(drop=True), selected_game.reset_index(drop=True)], axis=1)
    test_instance = test_instance[column_order]
    predictions = model.predict(test_instance) > 0.5
    
    # define 
    def Xplain_predict(X):
        return model.predict_proba(X)[:, 1]
    # build xplainer
    explainer = anchor_tabular.AnchorTabularExplainer(
        class_names=['DISLIKE', 'LIKE'],
        feature_names=X_train.columns.tolist(),
        data=X_train.values,  
        categorical_names={}  # Provide dictionary mapping categorical column indices to lists of category names, if any
    )
    
    test_instance = test_instance.values.reshape(1,-1)
    explanation = explainer.explain_instance(test_instance[0], lgbm_predict_fn, threshold=0.95)

    # Print the explanation details
    print('Anchor: %s' % ' AND '.join(explanation.names()))
    print('Precision: %.2f' % explanation.precision())
    print('Coverage: %.2f' % explanation.coverage())

    
    return predictions

In [78]:
column_order = X_train.columns.tolist()
dataset = pd.concat([X_train,X_test])
new_user = dataset.drop_duplicates(subset='uid').loc[:, ['uid', 'Cluster']]
new_game = dataset.drop_duplicates(subset='BGGId').drop(columns=['uid', 'Cluster'])
#pick random user,game
random_user = new_user['uid'].sample(n=1).iloc[0]
random_game = new_game['BGGId'].sample(n=1).iloc[0]
# filter dataframes
selected_user = new_user[new_user['uid'] == random_user]
selected_game = new_game[new_game['BGGId'] == random_game]
test_instance = pd.concat([selected_user.reset_index(drop=True), selected_game.reset_index(drop=True)], axis=1)
test_instance = test_instance[column_order]
predictions = trained_model.predict(test_instance) > 0.5



# Predict function for the Anchor explainer (probability of the positive class)
predict_fn = lambda x: trained_model.predict_proba(x)[:, 1]

# Initialize the explainer with feature names
explainer = AnchorTabular(predict_fn, feature_names=X_train.columns.tolist())

explainer.fit(X_train.values)

test_instance = test_instance.values.reshape(1, -1)

# Generate an explanation
explanation = explainer.explain(test_instance, threshold=0.95)

# Print the explanation
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Anchor: NumExpansions <= 0.00 AND War > 0.00 AND 1.00 < NumAlternates <= 3.00 AND CGS <= 0.00 AND Strategy <= 0.00 AND Party <= 0.00 AND BGGId <= 15878.00 AND 1.00 < GameWeight <= 2.00 AND 1.00 < MfgPlaytime <= 2.00 AND Abstract <= 0.00 AND NumImplementations <= 0.00 AND -1.00 < Cluster <= 0.00 AND uid > 231400.00 AND Family <= 0.00 AND Thematic <= 0.00 AND Kickstarted <= 0.00 AND Childrens <= 0.00
Precision: 0.15
Coverage: 0.00
